In [18]:
# Does numba work with annoy
import numpy as np
from annoy import AnnoyIndex
from numba import jit

@jit(fastmath=True, forceobj=True)
def test_numba():
    f = 40
    t = AnnoyIndex(f, 'euclidean')
    for i in range(10000):
        v = np.random.rand(f)
        t.add_item(i, v)
    t.build(10)
    for j in range(100000):
        t.get_nns_by_vector(np.random.rand(f), 10)

In [19]:
test_numba()

/tmp/ipykernel_996806/2664387536.py:6: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "test_numba" failed type inference due to: non-precise type pyobject
During: typing of argument at /tmp/ipykernel_996806/2664387536.py (10)

File "../../../../../tmp/ipykernel_996806/2664387536.py", line 10:
<source missing, REPL/exec in use?>

  @jit(fastmath=True, forceobj=True)
/tmp/ipykernel_996806/2664387536.py:6: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "test_numba" failed type inference due to: non-precise type pyobject
During: typing of argument at /tmp/ipykernel_996806/2664387536.py (13)

File "../../../../../tmp/ipykernel_996806/2664387536.py", line 13:
<source missing, REPL/exec in use?>

  @jit(fastmath=True, forceobj=True)


In [20]:
f = 40
t = AnnoyIndex(f, 'euclidean')
for i in range(10000):
    v = np.random.rand(f)
    t.add_item(i, v)
t.build(10)
for j in range(100000):
    t.get_nns_by_vector(np.random.rand(f), 10)

# Memory calculator

590.6 MiB for ann_idx and data
peak usage: 1034 MiB

In [4]:
195.6 / 2000000 * 1024 * 1024 / 3

34.1835776

In [18]:
10000 * 64 / 8 / 1024 / 1024 * 99 * 4

30.21240234375

In [13]:
100**3 * 64 / 8 / 1024 / 1024

7.62939453125

In [15]:
1000**3 * 64 / 8 / 1024 / 1024 * 3

22888.18359375

In [16]:
3442-2861

581

In [7]:
500**3 * 64 / 8 / 1024 / 1024

953.67431640625

In [6]:
1300/747, 5827/1300

(1.7402945113788488, 4.4823076923076925)

In [ ]:
import numpy as np
700 + 

In [10]:
3441 - 595, 1002 - 595

(2846, 407)